# Update Data
A couple different workflows for updating data:
* Apply edits -- good for small, transactional edits
* Overwrite -- good for ETL from another system
* TODO Append -- good for updating existing layers and ETL

In [1]:
import json
import tempfile
import os
import shutil
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`cool_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/cool_utils.py) and [`feature_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/feature_utils.py)

In [2]:
from partnerutils.cool_utils import chunk_df
from partnerutils.feature_utils import sdf_from_xyz

In [3]:
# this will prompt for a password
# can also do GIS("https://www.arcgis.com", "<USERNAME>", "<PASSWORD>")
gis = GIS(username="mpayson_startups")

Enter password:  ········


## "Apply Edits"
This uses the **[`edit_features`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.FeatureLayer.edit_features)** function to append data when working with a [Spatially Enabled DataFrame](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/). This method is best for small updates as it sends individual features as JSON and applies row-by-row edits. You can also update an existing feature using the `updates` parameter in `edit_features` by specifying a common Object ID.

In [4]:
# path to data
csv_path = "../sample_data/NYC_Restaurant_Inspections.csv"
x_col = "Longitude"
y_col = "Latitude"

In [5]:
# read csv and construct spatial dataframe
df = pd.read_csv(csv_path)
sdf = sdf_from_xyz(df, x_col, y_col)
len(sdf)

5000

In [6]:
# iterate through chunks to create and append data
lyr = None
for c_df in chunk_df(sdf):
    if not lyr:
        item = c_df.spatial.to_featurelayer("MyFeatureService", tags="esri_partner_tools_sample")
        lyr = item.layers[0]
    else:
        # THIS IS THE APPEND DATA PART
        fs = c_df.spatial.to_featureset()
        success = lyr.edit_features(adds=fs)
item

<Item title:"MyFeatureService" type:Feature Layer Collection owner:mpayson_startups>

## "Overwrite"
This uses the **[`ovewrite`](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html?highlight=overwrite#arcgis.features.managers.FeatureLayerCollectionManager.overwrite)** method. It deletes all data in an existing service and replaces it with updated data from the specified file. This is the most efficient method if you are completely updating a service.

Note, the overwriting file type **must be the same** as the type used to create the initial service (more info in doc). If you publish from a data frame (like above) the Python API zips to a FGDB or SHP before publishing, so updating with geojson or CSV won't work.

In [7]:
# get two geojson subsets to showcase the workflow
file_path = "../sample_data/NYC_Restaurant_Inspections.geojson"
with open(file_path) as file:
    geojson = json.load(file)

features = geojson.pop('features')
init_geo = {**geojson, **{'features': features[0:3000]}}
next_geo = {**geojson, **{'features': features[3000:5000]}}

In [8]:
# publish the initial layer
temp_dir_path = tempfile.mkdtemp()
temp_file_path = os.path.join(temp_dir_path, 'Inspections.geojson')
with open(temp_file_path, 'w') as temp_file:
    json.dump(init_geo, temp_file)

# add geojson as an item
item = gis.content.add({
    "type": "GeoJson",
    "title": "Inspections",
    "tags": "esri_partner_tools_sample"
}, data=temp_file_path)
shutil.rmtree(temp_dir_path)

# publish the geojson to a hosted feature layer
lyr_item = item.publish()
lyr_item

<Item title:"Inspections" type:Feature Layer Collection owner:mpayson_startups>

In [9]:
# overwrite the layer with second geojson 
temp_dir_path = tempfile.mkdtemp()
temp_file_path = os.path.join(temp_dir_path, 'Inspections.geojson')
with open(temp_file_path, 'w') as temp_file:
    json.dump(next_geo, temp_file)

# THIS IS THE IMPORTANT PART
flc = FeatureLayerCollection.fromitem(lyr_item)
flc.manager.overwrite(temp_file_path)

shutil.rmtree(temp_dir_path)

lyr_item

<Item title:"Inspections" type:Feature Layer Collection owner:mpayson_startups>

## TODO "Append" (PR's _greatly_ apppreciated)!
This uses the **[`append`](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.append)** method (exclusive to ArcGIS Online). It upserts data from a previously existing file item and is more efficient for larger updates. It can also update existing features. By default, this uses the Object ID attribute to determine which features to update, but you can also specify your own field, this field just [has to be indexed](https://github.com/mpayson/esri-partner-tools/blob/master/feature_layers/manage_indexes.ipynb) in ArcGIS Online.

In [10]:
# todo

### Clean up

In [11]:
delete_items = gis.content.search("tags:esri_partner_tools_sample")
gis.content.delete_items(delete_items)

True